# BUILD A REDUCED ORDER MODEL

### Imports and custom class definitions

In [ ]:
import os
import io
import sys
import time
try:
    import mfem.par as mfem
except ModuleNotFoundError:
    msg = "PyMFEM is not installed yet. Install PyMFEM:\n"
    msg += "\tgit clone https://github.com/mfem/PyMFEM.git\n"
    msg += "\tcd PyMFEM\n"
    msg += "\tpython3 setup.py install --with-parallel\n"
    raise ModuleNotFoundError(msg)

from ctypes import c_double
from mfem.par import intArray
import numpy as np
from numpy import sin, cos, exp, sqrt, pi, abs, array, floor, log, sum

In [ ]:
sys.path.append("../../build")
import pylibROM.linalg as libROM
from pylibROM.mfem import ComputeCtAB

In [ ]:
from stopwatch import StopWatch

#### By construct pylibROM is capable of handling parallel computations

In [ ]:
from mpi4py import MPI
comm = MPI.COMM_WORLD
myid = comm.Get_rank()
num_procs = comm.Get_size()

#### Parameters

In [ ]:
frequency = 1.6666666666666665
id = 3

In [ ]:
rom_data_dir = "rom_data/"
train_data_dir = "training_data/"

#### Specifying arguments for MFEM

In [ ]:
from parser_config import get_parser
parser = get_parser()

# Online phase:
args = parser.parse_args(f"-online -f {frequency} -id {id}".split())

In [ ]:
parser.print_options(args)

freq            = args.frequency
fom             = args.fom
offline         = args.offline
online          = args.online
merge           = args.merge
device_config   = args.device
id              = args.id
order           = args.order
nsets           = args.nset
coef            = args.coefficient
pa              = args.partial_assembly
static_cond     = args.static_condensation
visualization   = args.visualization
precision       = 8
paraview        = args.paraview

kappa = np.pi*freq

#### Enable hardware devices such as GPUs, and programming models such as CUDA, OCCA, RAJA and OpenMP based on command line options.

In [ ]:
device = mfem.Device(device_config)
if (myid == 0):
    device.Print()

In [ ]:
mesh_file = train_data_dir+"mesh.000000"
mesh = mfem.Mesh(mesh_file, 0, 0) #No modifications
dim = mesh.Dimension()

In [ ]:
pmesh = mfem.ParMesh(comm, mesh)
mesh.Clear()

#### Define a parallel finite element space on the parallel mesh. Here we use continuous Lagrange finite elements of the specified order. If order < 1, we instead use an isoparametric/isogeometric space.**

In [ ]:
if (order > 0):
    fec = mfem.H1_FECollection(order, dim)
    delete_fec = True
elif (pmesh.GetNodes()):
    fec = pmesh.GetNodes().OwnFEC()
    delete_fec = False
    if (myid == 0):
        print("Using isoparametric FEs: %s" % fec.Name())
else:
    fec = mfem.H1_FECollection(1, dim)
    delete_fec = True

fespace = mfem.ParFiniteElementSpace(pmesh, fec)
size = fespace.GlobalTrueVSize()
if (myid == 0):
    print("Number of finite element unknowns: %d" % size)

#### Determine the list of true (i.e. parallel conforming) essential  boundary dofs. In this example, the boundary conditions are defined  by marking all the boundary attributes from the mesh as essential  (Dirichlet) and converting them to a list of true dofs.**

In [ ]:
ess_tdof_list = mfem.intArray()
if (pmesh.bdr_attributes.Size() > 0):
    ess_bdr = mfem.intArray(pmesh.bdr_attributes.Max())
    ess_bdr.Assign(1)
    fespace.GetEssentialTrueDofs(ess_bdr, ess_tdof_list)

In [ ]:
basisName = train_data_dir+"basis"
basisFileName = "%s%d" % (basisName, id)
solveTimer, assembleTimer = StopWatch(), StopWatch()

#### Set up the parallel linear form b(.) which corresponds to the right-hand side of the FEM linear system, which in this case is (f,phi_i) where f is given by the function f_exact and phi_i are the basis functions in the finite element fespace.

In [ ]:
assembleTimer.Start()
b = mfem.ParLinearForm(fespace)
class RightHandSide(mfem.PyCoefficient):
    def EvalValue(self, x):
        if (dim == 3):
            return sin(kappa * (x[0] + x[1] + x[2]))
        else:
            return sin(kappa * (x[0] + x[1]))
f = RightHandSide()
b.AddDomainIntegrator(mfem.DomainLFIntegrator(f))
b.Assemble()

#### Define the solution vector x as a parallel finite element grid function corresponding to fespace. Initialize x with initial guess of zero, which satisfies the boundary conditions.**


In [ ]:
x = mfem.ParGridFunction(fespace)
x.Assign(0.0)

#### Set up the parallel bilinear form a(.,.) on the finite element space corresponding to the Laplacian operator -Delta, by adding the Diffusion domain integrator

In [ ]:
a = mfem.ParBilinearForm(fespace)
one = mfem.ConstantCoefficient(coef)
if (pa):
    a.SetAssemblyLevel(mfem.AssemblyLevel_PARTIAL)
a.AddDomainIntegrator(mfem.DiffusionIntegrator(one))

#### Assemble the parallel bilinear form and the corresponding linear   system, applying any necessary transformations such as: parallel   assembly, eliminating boundary conditions, applying conforming   constraints for non-conforming AMR, static condensation, etc.

In [ ]:
if (static_cond):
    a.EnableStaticCondensation()
a.Assemble()

A = mfem.HypreParMatrix()
B = mfem.Vector()
X = mfem.Vector()
a.FormLinearSystem(ess_tdof_list, x, b, A, X, B)
assembleTimer.Stop()

In [ ]:
if (online):
    # 20. read the reduced basis
    assembleTimer.Start()
    reader = libROM.BasisReader(basisName)
    spatialbasis = reader.getSpatialBasis(0.0)
    numRowRB = spatialbasis.numRows()
    numColumnRB = spatialbasis.numColumns()
    if (myid == 0):
        print("spatial basis dimension is %d x %d\n" % (numRowRB, numColumnRB))

    # libROM stores the matrix row-wise, so wrapping as a DenseMatrix in MFEM means it is transposed.
    reducedBasisT = mfem.DenseMatrix(spatialbasis.getData())

    # 21. form inverse ROM operator
    invReducedA = libROM.Matrix(numColumnRB, numColumnRB, False)
    ComputeCtAB(A, spatialbasis, spatialbasis, invReducedA)
    invReducedA.invert()

    bData = np.array((c_double * B.Size()).from_address(int(B.GetData())), copy=False)
    B_carom = libROM.Vector(bData, True, False)
    xData = np.array((c_double * X.Size()).from_address(int(X.GetData())), copy=False)
    X_carom = libROM.Vector(xData, True, False)
    reducedRHS = spatialbasis.transposeMult(B_carom)
    reducedSol = libROM.Vector(numColumnRB, False)
    assembleTimer.Stop()

    # 22. solve ROM
    solveTimer.Start()
    invReducedA.mult(reducedRHS, reducedSol)
    solveTimer.Stop()

    # 23. reconstruct FOM state
    spatialbasis.mult(reducedSol, X_carom)
    del spatialbasis
    del reducedRHS

#### Recover the parallel grid function corresponding to X. This is the  local finite element solution on each processor

In [ ]:
a.RecoverFEMSolution(X, b, x)

#### Stringstream sol_dofs_name, sol_dofs_name_fom

In [ ]:
import os
if not os.path.exists('rom_data'):
    os.makedirs('rom_data')

In [ ]:
if (online):
    sol_dofs_name = rom_data_dir+f"rsol_{id}_"+"dofs.%06d" % myid
    sol_dofs_name_fom = rom_data_dir+f"sol_{id}_"+"dofs_fom.%06d" % myid

    # Initialize FOM solution
    x_fom = mfem.Vector(x.Size())

    # Open and load file
    x_fom.Load(sol_dofs_name_fom, x_fom.Size())

    diff_x = mfem.Vector(x.Size())

    mfem.subtract_vector(x, x_fom, diff_x)

    # Get norms
    tot_diff_norm = np.sqrt(mfem.InnerProduct(comm, diff_x, diff_x))
    tot_fom_norm = np.sqrt(mfem.InnerProduct(comm, x_fom, x_fom))

    if (myid == 0):
        print("Relative error of ROM solution = %.5E" % (tot_diff_norm / tot_fom_norm))

#### Save the refined mesh and the solution in parallel. This output can be viewed later using GLVis: "glvis -np <np> -m mesh -g sol"

In [ ]:
mesh_name  = rom_data_dir+"mesh.%06d" % myid
sol_name = rom_data_dir+"sol.%06d" % myid

pmesh.Print(mesh_name, precision)

output = io.StringIO()
output.precision = precision
x.Save(output)

# with open(sol_name, 'wb') as file:
#     file.write(output.getvalue())

fid = open(sol_name, 'w')
fid.write(output.getvalue())
fid.close()

xData = np.array((c_double * X.Size()).from_address(int(X.GetData())), copy=False)
np.savetxt(sol_dofs_name, xData, fmt='%.16f')

#### Print timing info

In [ ]:
if (myid == 0):
    if (fom or offline):
        print("Elapsed time for assembling FOM: %e second\n" % assembleTimer.duration)
        print("Elapsed time for solving FOM: %e second\n" % solveTimer.duration)


    if(online):
        print("Elapsed time for assembling ROM: %e second\n" % assembleTimer.duration)
        print("Elapsed time for solving ROM: %e second\n" % solveTimer.duration)


#### Save data to file

In [ ]:
# Define the file name where you want to write the times
filename_ROM = rom_data_dir+'Assemble_solve_ROM_log.csv'

if myid == 0:
    # Open the file in append mode
    if online:
        with open(filename_ROM, 'a') as file:
            # Write assembly and solve times for ROM to the file in a structured format
            file.write("ROM,Assemble,%e\n" % assembleTimer.duration)
            file.write("ROM,Solve,%e\n" % solveTimer.duration)
            file.write("ROM,Accuracy,%e\n" % (tot_diff_norm / tot_fom_norm))


#### Paraview

In [ ]:
if paraview:
    paraview_dc = mfem.ParaViewDataCollection(f"Poisson_pv_{id}", pmesh)
    paraview_dc.SetPrefixPath(rom_data_dir+"ParaView_online")
    paraview_dc.SetLevelsOfDetail(order)
    paraview_dc.SetCycle(0)
    paraview_dc.SetDataFormat(mfem.VTKFormat_BINARY)
    paraview_dc.SetHighOrderOutput(True)
    paraview_dc.SetTime(0.0)
    paraview_dc.RegisterField(f"solution_{id}", x)
    paraview_dc.Save()

#### Free the used memory

In [ ]:
if (delete_fec):
    del fec
MPI.Finalize()